# Starting Analysis of Simulation Output - ROV, ASPA, PeerROV

## So far the scenarios and policies for ROV, ASPA & PeerROV have finished, still working on ROV++
- Dataset is found at `./dataAnalysis/consolidatedData.csv`

## Activate Environment

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("CairoMakie")

## Necessary Imports

In [ ]:
using CSV
using DataFrames
using Statistics
using Plots
using CairoMakie

## Set var for csv path

In [ ]:
csv_path = "./dataAnalysis/consolidatedData.csv"

## Seeing which deployment type for any policy against any scenario works best

In [ ]:
function calculate_victim_success_rate(csv_path::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    all_results = DataFrame()

    𝒫, 𝒮 = unique(𝒟.AdoptingPolicyCls), unique(𝒟.scenario_cls)
    for policy in 𝒫
        for scenario in 𝒮
            if (policy == "ROVPPV2Lite" || policy == "ROVPPV1Lite" || policy == "ROVPPV2ImprovedLite") && scenario == "AccidentalRouteLeak"
                continue
            end
            𝒟ᶠ = filter(row -> row.AdoptingPolicyCls == policy && row.scenario_cls == scenario && row.outcome == "VICTIM_SUCCESS", 𝒟) 

            𝒢 = groupby(𝒟ᶠ, :deployment_type)
            𝑅 = combine(𝒢, :value => mean => :victim_success_rate)
            
            𝑅[!, :AdoptingPolicyCls] .= policy
            𝑅[!, :scenario_cls] .= scenario

            all_results = vcat(all_results, 𝑅)
        end
    end
    all_results = select(all_results, :AdoptingPolicyCls, :scenario_cls, :deployment_type, :victim_success_rate)

    return sort(all_results, :victim_success_rate, rev=true)
end

println(calculate_victim_success_rate(csv_path))

## Bar graph for deployment type variability for all scenarios and policies

In [ ]:
function barGraph_𝑣ₛᵣ(csv_path::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    graphs_dir = "./barGraphs"
    isdir(graphs_dir) || mkdir(graphs_dir)

    𝒫, 𝒮 = unique(𝒟.AdoptingPolicyCls), unique(𝒟.scenario_cls)

    𝒞 = Dict(zip(𝒫, [:yellow, :orange, :pink, :cyan, :magenta, :yellow, :navajowhite]))

    for policy in 𝒫
       𝒟ₜ = Dict{String, Vector{Any}}(
        "NoDeploymentType" => [],
        "INPUT_CLIQUE" => [],
        "MULTIHOMED" => [],
        "STUBS" => [],
       )
        for scenario in 𝒮
            if (policy == "ROVPPV2Lite" || policy == "ROVPPV1Lite" || policy == "ROVPPV2ImprovedLite") && scenario == "AccidentalRouteLeak"
                continue
            end

            𝒟ᶠ = filter(row -> row.AdoptingPolicyCls == policy && row.scenario_cls == scenario && row.outcome == "VICTIM_SUCCESS", 𝒟)

            𝒢 = groupby(𝒟ᶠ, :deployment_type)
            𝑣ₛᵣ𝒟 = combine(𝒢, :value => mean => :𝑣ₛᵣ)

            for row in eachrow(𝑣ₛᵣ𝒟)
                append!(𝒟ₜ[row.deployment_type], row.𝑣ₛᵣ)
            end

            𝑦ₘᵢₙ, 𝑦ₘₐₓ = minimum(𝑣ₛᵣ𝒟.𝑣ₛᵣ), maximum(𝑣ₛᵣ𝒟.𝑣ₛᵣ)
            𝑦ₗₘᵢₙ, 𝑦ₗₘₐₓ = max(0, 𝑦ₘᵢₙ - 5), min(100, 𝑦ₘₐₓ + 15)

            bar_plot = bar(𝑣ₛᵣ𝒟.deployment_type,
                           𝑣ₛᵣ𝒟.𝑣ₛᵣ,
                           label="Victim Success Rate",
                           title="Victim Success Rates for $policy\n$scenario",
                           ylabel="Success Rate (%)",
                           xlabel="Deployment Type",
                           legend=:best,
                           color=𝒞[policy],
                           xrotation=45,
                           margin=10Plots.mm)

            Plots.ylims!(𝑦ₗₘᵢₙ, 𝑦ₗₘₐₓ)

            for (i, val) in enumerate(𝑣ₛᵣ𝒟.𝑣ₛᵣ)
                if val ≥ 5
                    annotate!(bar_plot, [(i+0.5-1, val - 2, Plots.text(string(round(val, digits=3)), :center, 8))])
                else
                    annotate!(bar_plot, [(i+0.5-1, val + 2, Plots.text(string(round(val, digits=3)), :center, 8))])
                end
            end

            plot_filename = joinpath(graphs_dir, "$(policy)_$(scenario)_Victim_Success_Rates.png")
            savefig(bar_plot, plot_filename)

            # display(bar_plot)
        end
        
        𝑂ₘ = Dict{String, Float64}()
        for (deployment_type, rates) in 𝒟ₜ
            𝑂ₘ[deployment_type] = mean(rates)
        end

        𝑦ₒₘᵢₙ, 𝑦ₒₘₐₓ = minimum(values(𝑂ₘ)), maximum(values(𝑂ₘ))
        𝑦ₒₗₘᵢₙ, 𝑦ₒₗₘₐₓ = max(0, 𝑦ₒₘᵢₙ - 5), min(100, 𝑦ₒₘₐₓ + 15)

        overall_plot = bar(collect(keys(𝑂ₘ)),
                           collect(values(𝑂ₘ)),
                           label="Overall Victim Success Rate",
                           title="Overall Victim Success Rates for $policy",
                           ylabel="Success Rate (%)",
                           xlabel="Deployment Type",
                           legend=:best,
                           color=𝒞[policy],
                           xrotation=45,
                           margin=10Plots.mm)
        
        for (i, val) in enumerate(values(𝑂ₘ))
            if val ≥ 5
                annotate!(overall_plot, [(i+0.5-1, val - 2, Plots.text(string(round(val, digits=3)), :center, 8))])
            else 
                annotate!(overall_plot, [(i+0.5-1, val + 2, Plots.text(string(round(val, digits=3)), :center, 8))])
            end
        end

        Plots.ylims!(𝑦ₒₗₘᵢₙ, 𝑦ₒₗₘₐₓ)

        overall_plot_filename = joinpath(graphs_dir, "$(policy)_Overall_Victim_Success_Rates.png")
        savefig(overall_plot, overall_plot_filename)

        # display(overall_plot)
    end
end

barGraph_𝑣ₛᵣ(csv_path)
println("Oh 🍯")

## Bar Graphs by Scenario

In [ ]:
function scenarioBarGraph_𝑣ₛᵣ(csv_path::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    graphs_dir = "./scenarioBarGraphs"
    isdir(graphs_dir) || mkdir(graphs_dir)

    𝒫, 𝒮 = unique(𝒟.AdoptingPolicyCls), unique(𝒟.scenario_cls)
    𝒞 = Dict(zip(𝒫, [:yellow, :orange, :pink, :cyan, :magenta, :skyblue, :navajowhite]))

    for scenario in 𝒮
        policy_data = Dict{String, Dict{String, Any}}()

        for policy in 𝒫
            if (policy == "ROVPPV2Lite" || policy == "ROVPPV1Lite" || policy == "ROVPPV2ImprovedLite") && scenario == "AccidentalRouteLeak"
                continue
            end
            𝒟ᶠ = filter(row -> row.AdoptingPolicyCls == policy && row.scenario_cls == scenario && row.outcome == "VICTIM_SUCCESS", 𝒟)
            𝒢 = groupby(𝒟ᶠ, :deployment_type)
            𝑣ₛᵣ𝒟 = combine(𝒢, :value => mean => :𝑣ₛᵣ)

            if !haskey(policy_data, policy)
                policy_data[policy] = Dict("Overall" => [], "Details" => Dict{String, Float64}())
            end
            if !isempty(𝑣ₛᵣ𝒟.𝑣ₛᵣ)
                policy_data[policy]["Overall"] = mean(𝑣ₛᵣ𝒟.𝑣ₛᵣ)
            end
            
            for row in eachrow(𝑣ₛᵣ𝒟)
                policy_data[policy]["Details"][row.deployment_type] = row.𝑣ₛᵣ
            end
        end

        x_labels, values, colors = [], [], []

 
        for (policy, data) in policy_data
            push!(x_labels, "$policy Overall")
            push!(values, data["Overall"])
            push!(colors, 𝒞[policy])
            for (dtype, rate) in data["Details"]
                push!(x_labels, "$policy $dtype")
                push!(values, rate)
                push!(colors, 𝒞[policy])
            end
        end
        xtick_positions = 1:length(x_labels)
        offset = 0.5
        adjusted_xtick_positions = xtick_positions .- offset

        bar_plot = bar(
            x_labels, 
            values, 
            label="", 
            title="Victim Success Rates Across Policies for Scenario\n$scenario", 
            ylabel="Success Rate (%)", 
            xlabel="Policies and Deployment Types",
            legend=false, 
            color=colors, 
            xrotation=45, 
            margin=10Plots.mm,
            size=(1900,1400),
            xlims=(0, length(x_labels) + 1),
            xticks=(adjusted_xtick_positions, x_labels)
        )

        plot_filename = joinpath(graphs_dir, "$scenario-Victim_Success_Rates.png")
        savefig(bar_plot, plot_filename)
        # display(bar_plot)
    end
end

scenarioBarGraph_𝑣ₛᵣ(csv_path)

## Graph Constants

In [ ]:
const DEPLOYMENT_TYPE_MAP = Dict(
    "STUBS" => 1,
    "MULTIHOMED" => 2,
    "INPUT_CLIQUE" => 3,
    "NoDeploymentType" => 4
)
const DEPLOYMENT_TYPE_NAMES = Dict(
    1 => "STUBS",
    2 => "MULTIHOMED",
    3 => "INPUT_CLIQUE",
    4 => "NoDeploymentType"
)
const PERCENT_ADOPT_LABELS = ["1e-6", "0.1", "0.2", "0.4", "0.8", "0.99"]


## Data Processing Function

In [ ]:
function process_data(csv_path::String, save_dir::String, processing_function::Function)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒫, 𝒮 = unique(𝒟.AdoptingPolicyCls), unique(𝒟.scenario_cls)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")

    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)

    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]

    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    for policy in 𝒫
        for scenario in 𝒮
            if (policy == "ROVPPV2Lite" || policy == "ROVPPV1Lite" || policy == "ROVPPV2ImprovedLite") && scenario == "AccidentalRouteLeak"
                continue
            end
            
            policy_str, scenario_str = String(policy), String(scenario)
            filtered_𝒟 = 𝒟[(𝒟.AdoptingPolicyCls .== policy) .& (𝒟.scenario_cls .== scenario), :]

            processing_function(filtered_𝒟, policy_str, scenario_str, save_dir)
        end
    end
end

## 2D heat maps for adoption rate vs deployment type vs success rate

In [ ]:
function generate_2d_heatmap(𝒟::DataFrame, policy::String, scenario::String, heat_maps_dir::String)
    if nrow(𝒟) == 0
        println("No data for policy $policy and scenario $scenario")
        return
    end

    agg_𝒟 = combine(groupby(𝒟, [:percent_adopt, :deployment_type]), :value => mean)

    𝑝 = agg_𝒟.percent_adopt
    deployment_type = agg_𝒟.deployment_type
    victim_success_rate = agg_𝒟.value_mean

    𝑝 = Float64[𝑝...]
    deployment_type = Float64[deployment_type...]
    victim_success_rate = Float64[victim_success_rate...]

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "$(policy) - $(scenario)", xlabel = "Adoption Percentage", ylabel = "Deployment Type")
    hm = CairoMakie.heatmap!(ax, 𝑝, deployment_type, victim_success_rate, colormap = :thermal, colorrange = (minimum(victim_success_rate), maximum(victim_success_rate)))
    
    ax.yticks = (1:4, ["STUBS", "MULTIHOMED", "INPUT_CLIQUE", "NoDeploymentStrategy"])

    ax.xticks = ([1e-6, 0.1, 0.2, 0.4, 0.8, 0.99], PERCENT_ADOPT_LABELS)

    Colorbar(fig[1, 2], hm, label = "Victim Success Rate (%)")

    save(joinpath(heat_maps_dir, "$(policy)_$(scenario)_2d_heatmap.png"), fig)

end
process_data(csv_path, "./heatmaps", generate_2d_heatmap)


## Crossbar Visualization to include y-error

In [ ]:
function crossbar_plot(𝒟::DataFrame, policy::String, scenario::String, graphs_dir::String)
    if nrow(𝒟) == 0
        println("No data for policy $policy and scenario $scenario")
        return
    end

    deployment_types = unique(𝒟.deployment_type)
    fig = Figure(size = (800, 600), padding = (50, 50, 50, 50))
    ax = Axis(fig[1, 1], title = "Crossbar Plot: $(policy) - $(scenario)", xlabel = "Deployment Type", ylabel = "Victim Success Rate (%)")

    for deployment_type in deployment_types
        subset_𝒟 = 𝒟[𝒟.deployment_type .== deployment_type, :]
        if isempty(subset_𝒟)
            continue
        end
        
        μ, σ = mean(subset_𝒟.value), mean(subset_𝒟.yerr)
        ⊤, ⊥ = μ .+ 1.645 * σ, μ .- 1.645 * σ

        deployment_type_num = DEPLOYMENT_TYPE_MAP[DEPLOYMENT_TYPE_NAMES[deployment_type]]

        Makie.scatter!(ax, [deployment_type_num], [μ], color=:blue, markersize=10)
        lines!(ax, [deployment_type_num, deployment_type_num], [⊥, ⊤], color=:red, linewidth=21)

        text!(ax, "⊥", position = (deployment_type_num, ⊥ - 2), color = :black, align = (:center, :top))
        text!(ax, "⊤", position = (deployment_type_num, ⊤ + 2), color = :black, align = (:center, :bottom))
    end
    ax.xticks = (1:4, ["STUBS", "MULTIHOMED", "INPUT_CLIQUE", "No\nDeployment\nType"])

    save(joinpath(graphs_dir, "$(policy)_$(scenario)_crossbar.png"), fig)
    # display(fig)
end
process_data(csv_path, "./crossbarGraphs", crossbar_plot)

## Correlation Matrix for Heatmaps

In [ ]:
function generate_correlation_heatmap(𝒟::DataFrame, policy::String, scenario::String, heat_maps_dir::String)
    if nrow(𝒟) == 0
        println("No data for policy $policy and scenario $scenario")
        return
    end

    numeric_df = select(𝒟, [:percent_adopt, :value])
    cor_matrix = cor(Matrix(numeric_df))

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "Correlation Matrix: $(policy) - $(scenario)", xlabel = "Variables", ylabel = "Variables")
    hm = CairoMakie.heatmap!(ax, cor_matrix, colormap = :thermal, colorrange = (-1, 1))
    
    ax.xticks = (1:2, ["percent_adopt", "Victim Success Rate"])
    ax.yticks = (1:2, ["percent_adopt", "Victim Success Rate"])

    Colorbar(fig[1, 2], hm, label = "Correlation")

    save(joinpath(heat_maps_dir, "$(policy)_$(scenario)_correlation_heatmap.png"), fig)
end
process_data(csv_path, "./corrMatrixHeatmaps", generate_correlation_heatmap)

## Facet Grid

In [ ]:
function generate_facet_grid(𝒟::DataFrame, policy::String, scenario::String, facet_dir::String)
    if nrow(𝒟) == 0
        println("No data for policy $policy and scenario $scenario")
        return
    end

    fig = Figure(size = (1200, 800), figure_padding=50)


    layout = fig[1, 1] = GridLayout()

    colgap!(layout, 150)

    for (i, dt) in enumerate(DEPLOYMENT_TYPE_MAP)
        filtered_𝒟 = 𝒟[𝒟.deployment_type .== dt[2], :]
        
        if isempty(filtered_𝒟)
            continue
        end

        ax = Axis(layout[1, i], title = "$(policy) \n $(scenario) \n $(dt[1])", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate")
        Makie.scatter!(ax, filtered_𝒟.percent_adopt, filtered_𝒟.value, color = :blue, markersize = 10)
    end

    save(joinpath(facet_dir, "$(policy)_$(scenario)_facet_grid.png"), fig)
    # display(fig)
end
process_data(csv_path, "./facet_grids", generate_facet_grid)

## Multiline Graph

In [ ]:
function generate_multiline_graphs(𝒟::DataFrame, policy::String, scenario::String, save_dir::String)
    if nrow(𝒟) == 0
        println("No data for policy $policy and scenario $scenario")
        return
    end

    agg_𝒟 = combine(groupby(𝒟, [:percent_adopt, :deployment_type]), :value => mean)

    deployment_types = unique(𝒟.deployment_type)
    percent_adopts = unique(agg_𝒟.percent_adopt)
    lines_data = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

    for deployment_type in deployment_types
        dt_name = DEPLOYMENT_TYPE_NAMES[deployment_type]
        dt_data = filter(row -> row.deployment_type == deployment_type, agg_𝒟)
        x = dt_data.percent_adopt
        y = dt_data.value_mean
        lines_data[dt_name] = (x, y)
    end

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "$(policy) - $(scenario)", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

    for (dt_name, (x, y)) in lines_data
        lines!(ax, x, y, label = dt_name)
    end

    Legend(fig[1, 2], ax, orientation = :vertical, title = "Deployment Types")

    save(joinpath(save_dir, "$(policy)_$(scenario)_multiline_graph.png"), fig)
end
process_data(csv_path, "./multiline_graphs", generate_multiline_graphs)

## Summary Statistics

In [ ]:
function calculate_descriptive_statistics(csv_path::String)
    df = CSV.read(csv_path, DataFrame)

    df_filtered = filter(row -> row.outcome == "ATTACKER_SUCCESS", df)

    df_filtered.percent_adopt .= replace(df_filtered.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    
    for col in [:percent_adopt, :value, :yerr]
        if eltype(df_filtered[!, col]) <: String
            df_filtered[!, col] = parse.(Float64, df_filtered[!, col])
        end
    end
    
    columns_stats = [:percent_adopt, :value, :yerr]

    groups = groupby(df_filtered, [:AdoptingPolicyCls, :deployment_type, :scenario_cls])

    statistics = Dict{String, Any}()
    
    for (key, group) in pairs(groups)
        group_stats = Dict{String, Any}()
        for column in columns_stats
            column_data = group[!, column]

            stats = Dict(
                "mean" => mean(column_data),
                "median" => median(column_data),
                "std" => std(column_data),
                "min" => minimum(column_data),
                "max" => maximum(column_data)
            )

            group_stats[string(column)] = stats
        end
        statistics[string(key)] = group_stats
    end

    return statistics
end
function display_statistics(statistics::Dict{String, Any})
    for (key, group_stats) in statistics
        println("Group: ", key) 
        println("---------------------------------------------------")

        for (column, stats) in group_stats
            println("Column: ", column) 
            println("  Mean: ", stats["mean"])
            println("  Median: ", stats["median"])
            println("  Standard Deviation: ", stats["std"])
            println("  Minimum: ", stats["min"])
            println("  Maximum: ", stats["max"])
            println("---------------------------------------------------")
        end
        println("\n")
    end
end
function write_statistics_to_file(statistics::Dict{String, Any}, file_path::String)
    open(file_path, "w") do file
        for (key, group_stats) in statistics
            write(file, "Group: $key\n")
            write(file, "---------------------------------------------------\n")

            for (column, stats) in group_stats
                write(file, "Column: $column\n")
                write(file, "  Mean: $(stats["mean"])\n")
                write(file, "  Median: $(stats["median"])\n")
                write(file, "  Standard Deviation: $(stats["std"])\n")
                write(file, "  Minimum: $(stats["min"])\n")
                write(file, "  Maximum: $(stats["max"])\n")
                write(file, "---------------------------------------------------\n")
            end
            write(file, "\n")
        end
    end
end

function find_and_print_max_values(csv_path::String)
    df = CSV.read(csv_path, DataFrame)

    df_filtered = filter(row -> row.outcome == "VICTIM_SUCCESS", df)

    for col in [:value]
        if eltype(df_filtered[!, col]) <: String
            df_filtered[!, col] = parse.(Float64, df_filtered[!, col])
        end
    end

    groups = groupby(df_filtered, :scenario_cls)

    for (scenario, group) in pairs(groups)
        max_value = maximum(group.value)
        print("$max_value:$scenario")
        max_row = filter(row -> row.value == max_value, group)[1, :]
        println("Scenario: ", scenario)
        println("  Policy: ", max_row.AdoptingPolicyCls)
        println("  Deployment Type: ", max_row.deployment_type)
        println("  Percent Adopt: ", max_row.percent_adopt)
        println("  Max Value: ", max_row.value)
        println("---------------------------------------------------")
    end
end

output_dict = calculate_descriptive_statistics(csv_path)

# display_statistics(output_dict)
write_statistics_to_file(output_dict, "./descriptiveStats.txt")

find_and_print_max_values(csv_path)

## Tarball Creations: Final

In [ ]:
function create_tarball(output_filename::String, directories::Vector{String})
    existing_directories = filter(isdir, directories)

    tar_command = `tar -czf $(output_filename) $(directories)`

    try
        run(tar_command)
        println("Tarball '$(output_filename)' created successfully.")
    catch e
        println("Failed to create tarball: ", e)
    end
end

# Built for unix system with tar installed

directories = [
    "./facet_grids/",
    "./heatmaps/",
    "./corrMatrixHeatmaps/",
    "./crossbarGraphs/",
    "./scenarioBarGraphs/",
    "./barGraphs/",
    "./multiline_graphs/"
]

create_tarball("Visualizations.tar.gz", directories)

println("Tarball 'Visualizations.tar.gz' created successfully with the specified directories.")


# Hypothesis Testing

## Hypothesis 1
### Higher adoption % by any policy & deployment combo should lead to a greater success rate

Prove with:
- Heatmap that works for overall scenarios and deployment types but has policies as y values and deployment percentages as X-Values
- Multi-line graph for overall success rate of each policy vs adoption %

In [ ]:
function generate_overall_heatmap(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]

    policies = ["ROV", "PeerROV", "ASPA", "AS-Cones", "ROVPPV2Lite", "ROVPPV1Lite", "ROVPPV2ImprovedLite"]
    scenarios = unique(𝒟.scenario_cls)
    adoption_percentages = unique(𝒟.percent_adopt)

    fig = Figure(resolution = (1800, 1200))

    heatmaps = []

    for (idx, policy) in enumerate(policies)
        if policy in ["ROVPPV2Lite", "ROVPPV1Lite", "ROVPPV2ImprovedLite"]
            relevant_scenarios = filter(s -> s != "AccidentalRouteLeak", scenarios)
        else
            relevant_scenarios = scenarios
        end

        agg_𝒟 = combine(groupby(𝒟[𝒟.AdoptingPolicyCls .== policy, :], [:percent_adopt, :scenario_cls]), :value => mean)

        heatmap_matrix = Matrix{Float64}(undef, length(adoption_percentages), length(relevant_scenarios))

        for i in 1:length(relevant_scenarios)
            for j in 1:length(adoption_percentages)
                row = filter(r -> r.scenario_cls == relevant_scenarios[i] && r.percent_adopt == adoption_percentages[j], agg_𝒟)
                heatmap_matrix[j, i] = isempty(row) ? NaN : row[1, :value_mean]
            end
        end

        row_idx = (idx - 1) ÷ 3 + 1
        col_idx = (idx - 1) % 3 + 1
        if idx == 7  # Add the last heatmap to the end
            row_idx = 3
            col_idx = 2
        end

        ax = Axis(fig[row_idx, col_idx], title = "Victim Success Rate for $policy", xlabel = "Adoption Percentage", ylabel = "Scenarios")

        hm = CairoMakie.heatmap!(ax, adoption_percentages, 1:length(relevant_scenarios), heatmap_matrix, colormap = :thermal, colorrange = (minimum(skipmissing(vec(heatmap_matrix))), maximum(skipmissing(vec(heatmap_matrix)))))
        push!(heatmaps, hm)

        ax.xticks = ([1e-6, 0.1, 0.2, 0.4, 0.8, 0.99], ["1e-6", "0.1", "0.2", "0.4", "0.8", "0.99"])
        ax.yticks = (1:length(relevant_scenarios), relevant_scenarios)
    end

    colbar = Colorbar(fig[:, end+1], heatmaps[1], label = "Victim Success Rate (%)", height = Relative(1))

    save(joinpath(save_dir, "hypo1_1.png"), fig)
end

generate_overall_heatmap(csv_path, "./hypo")

In [ ]:
function generate_overall_multiline_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]

    agg_𝒟 = combine(groupby(𝒟, [:percent_adopt, :AdoptingPolicyCls]), :value => mean)

    policies = unique(𝒟.AdoptingPolicyCls)
    lines_data = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

    for policy in policies
        policy_data = filter(row -> row.AdoptingPolicyCls == policy, agg_𝒟)
        x = policy_data.percent_adopt
        y = policy_data.value_mean
        lines_data[String(policy)] = (x, y)
    end

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "Overall Victim Success Rate by Policy", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

    for (policy, (x, y)) in lines_data
        lines!(ax, x, y, label = policy)
    end

    Legend(fig[1, 2], ax, orientation = :vertical, title = "Policies")

    save(joinpath(save_dir, "hypo1_2.png"), fig)
end

generate_overall_multiline_graph(csv_path, "./hypo")

## Hypothesis 2
### ASPA will work best when deployed on Stubs against any strategy at any adoption %

Prove with: 
- Overall bar graph for ASPA
- Multi-line graph showing ASPA for all deployment types against overall attacks

In [ ]:
function generate_aspa_multiline_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    policy = "ASPA"
    𝒟 = 𝒟[𝒟.AdoptingPolicyCls .== policy, :]

    agg_𝒟 = combine(groupby(𝒟, [:percent_adopt, :deployment_type]), :value => mean)

    deployment_types = unique(𝒟.deployment_type)
    lines_data = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

    for deployment_type in deployment_types
        dt_name = DEPLOYMENT_TYPE_NAMES[deployment_type]
        dt_data = filter(row -> row.deployment_type == deployment_type, agg_𝒟)
        x = dt_data.percent_adopt
        y = dt_data.value_mean
        lines_data[dt_name] = (x, y)
    end

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "ASPA Policy - All Scenarios", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

    for (dt_name, (x, y)) in lines_data
        lines!(ax, x, y, label = dt_name)
    end

    Legend(fig[1, 2], ax, orientation = :vertical, title = "Deployment Types")

    save(joinpath(save_dir, "hypo2_2.png"), fig)
end
generate_aspa_multiline_graph(csv_path, "./hypo")

## Hypothesis 3
### PeerROV will work best when deployed on MultiHomed against any strategy at any adoption %

Prove with: 
- Overall bar graph for PeerROV
- Multi-line graph showing PeerROV for all deployment types against overall attacks

In [ ]:
function generate_peerROV_multiline_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    policy = "PeerROV"
    𝒟 = 𝒟[𝒟.AdoptingPolicyCls .== policy, :]

    agg_𝒟 = combine(groupby(𝒟, [:percent_adopt, :deployment_type]), :value => mean)

    deployment_types = unique(𝒟.deployment_type)
    lines_data = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

    for deployment_type in deployment_types
        dt_name = DEPLOYMENT_TYPE_NAMES[deployment_type]
        dt_data = filter(row -> row.deployment_type == deployment_type, agg_𝒟)
        x = dt_data.percent_adopt
        y = dt_data.value_mean
        lines_data[dt_name] = (x, y)
    end

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "PeerROV Policy - All Scenarios", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

    for (dt_name, (x, y)) in lines_data
        lines!(ax, x, y, label = dt_name)
    end

    Legend(fig[1, 2], ax, orientation = :vertical, title = "Deployment Types")

    save(joinpath(save_dir, "hypo3_2.png"), fig)
end
generate_peerROV_multiline_graph(csv_path, "./hypo")

## Hypothesis 4
### AS-Cones will work best when deployed on Input Clique against any strategy at any adoption %

Prove with: 
- Overall bar graph for AS-Cones
- Multi-line graph showing AS-Cones for all deployment types against overall attacks

In [ ]:
function generate_ascones_multiline_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    policy = "AS-Cones"
    𝒟 = 𝒟[𝒟.AdoptingPolicyCls .== policy, :]

    agg_𝒟 = combine(groupby(𝒟, [:percent_adopt, :deployment_type]), :value => mean)

    deployment_types = unique(𝒟.deployment_type)
    lines_data = Dict{String, Tuple{Vector{Float64}, Vector{Float64}}}()

    for deployment_type in deployment_types
        dt_name = DEPLOYMENT_TYPE_NAMES[deployment_type]
        dt_data = filter(row -> row.deployment_type == deployment_type, agg_𝒟)
        x = dt_data.percent_adopt
        y = dt_data.value_mean
        lines_data[dt_name] = (x, y)
    end

    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], title = "AS-Cones Policy - All Scenarios", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

    for (dt_name, (x, y)) in lines_data
        lines!(ax, x, y, label = dt_name)
    end

    Legend(fig[1, 2], ax, orientation = :vertical, title = "Deployment Types")

    save(joinpath(save_dir, "hypo4_2.png"), fig)
end
generate_ascones_multiline_graph(csv_path, "./hypo")

## Hypothesis 5
### ASPA will have highest victim success rate of any policy against Forged-Origin Prefix Hijacking 

Proves with:
- Bar graph that has only this deployment strategy & overall scenario success rate for all policies

In [ ]:
function generate_forged_origin_bar_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    scenario = "ForgedOriginPrefix"
    filtered_𝒟 = 𝒟[𝒟.scenario_cls .== scenario, :]
    𝒫 = unique(filtered_𝒟.AdoptingPolicyCls)

    policy_means = Dict{String, Float64}()

    for policy in 𝒫
        policy_data = filtered_𝒟[filtered_𝒟.AdoptingPolicyCls .== policy, :]
        overall_mean = mean(policy_data.value)
        policy_means[String(policy)] = overall_mean
    end

    x_labels = collect(keys(policy_means))
    y_values = collect(values(policy_means))

    bar_plot = bar(x_labels, y_values, legend = false, title = "Victim Success Rate for Forged-Origin Prefix Hijacking", xlabel = "Policies", ylabel = "Victim Success Rate (%)", color = :skyblue, xrotation=45)

    for (i, val) in enumerate(y_values)
        annotate!(bar_plot, (i-.5, val + 1, Plots.text(string(round(val, digits=3)), :black, 8)))
    end
    Plots.ylims!(50,80)

    savefig(joinpath(save_dir, "hypo5.png"))
end

generate_forged_origin_bar_graph(csv_path, "./hypo")

## Hypothesis 6
### ROV will have highest victim success rate of any policy against Prefix & SubPrefix Hijacking 

Proves with:
- Bar graph that has only this deployment strategy & overall scenario success rate for all policies

In [ ]:
function generate_prefix_bar_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    scenario = "PrefixHijack"
    filtered_𝒟 = 𝒟[𝒟.scenario_cls .== scenario, :]
    𝒫 = unique(filtered_𝒟.AdoptingPolicyCls)

    policy_means = Dict{String, Float64}()

    for policy in 𝒫
        policy_data = filtered_𝒟[filtered_𝒟.AdoptingPolicyCls .== policy, :]
        overall_mean = mean(policy_data.value)
        policy_means[String(policy)] = overall_mean
    end

    x_labels = collect(keys(policy_means))
    y_values = collect(values(policy_means))

    bar_plot = bar(x_labels, y_values, legend = false, title = "Victim Success Rate for Prefix Hijacking", xlabel = "Policies", ylabel = "Victim Success Rate (%)", color = :skyblue, xrotation=45)

    for (i, val) in enumerate(y_values)
        annotate!(bar_plot, (i-.5, val + 2, Plots.text(string(round(val, digits=3)), :black, 8)))
    end
    Plots.ylims!(0,100)

    savefig(joinpath(save_dir, "hypo6_1.png"))
end
generate_prefix_bar_graph(csv_path, "./hypo")


In [ ]:
function generate_subprefix_bar_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    scenario = "SubprefixHijack"
    filtered_𝒟 = 𝒟[𝒟.scenario_cls .== scenario, :]
    𝒫 = unique(filtered_𝒟.AdoptingPolicyCls)

    policy_means = Dict{String, Float64}()

    for policy in 𝒫
        policy_data = filtered_𝒟[filtered_𝒟.AdoptingPolicyCls .== policy, :]
        overall_mean = mean(policy_data.value)
        policy_means[String(policy)] = overall_mean
    end

    x_labels = collect(keys(policy_means))
    y_values = collect(values(policy_means))

    bar_plot = bar(x_labels, y_values, legend = false, title = "Victim Success Rate for SubPrefix Hijacking", xlabel = "Policies", ylabel = "Victim Success Rate (%)", color = :skyblue, xrotation=45)

    for (i, val) in enumerate(y_values)
        annotate!(bar_plot, (i-.6, val + 2.2, Plots.text(string(round(val, digits=3)), :black, 8)))
    end
    Plots.ylims!(0,100)

    savefig(joinpath(save_dir, "hypo6_2.png"))
end
generate_subprefix_bar_graph(csv_path, "./hypo")


## Hypothesis 7
### PeerROV will have highest victim success rate of any policy against Accidental Route Leaks

Proves with:
- Bar graph that has only this deployment strategy & overall scenario success rate for all policies

In [ ]:
function generate_arl_bar_graph(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)

    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]
    𝒟.deployment_type = map(x -> DEPLOYMENT_TYPE_MAP[x], 𝒟.deployment_type)

    scenario = "AccidentalRouteLeak"
    filtered_𝒟 = 𝒟[𝒟.scenario_cls .== scenario, :]
    𝒫 = unique(filtered_𝒟.AdoptingPolicyCls)

    policy_means = Dict{String, Float64}()

    for policy in 𝒫
        policy_data = filtered_𝒟[filtered_𝒟.AdoptingPolicyCls .== policy, :]
        overall_mean = mean(policy_data.value)
        policy_means[String(policy)] = overall_mean
    end

    x_labels = collect(keys(policy_means))
    y_values = collect(values(policy_means))

    bar_plot = bar(x_labels, y_values, legend = false, title = "Victim Success Rate for Accidental Route Leak", xlabel = "Policies", ylabel = "Victim Success Rate (%)", color = :skyblue, xrotation=45)

    for (i, val) in enumerate(y_values)
        annotate!(bar_plot, (i-.6, val + 2.2, Plots.text(string(round(val, digits=3)), :black, 8)))
    end
    Plots.ylims!(0,100)

    savefig(joinpath(save_dir, "hypo7.png"))
end


generate_arl_bar_graph(csv_path, "./hypo")

## Hypothesis 8
### Any policy, but particularly base ROV, will produce a higher victim success rate when using the Input Clique Deployment strategy than any other deployment strategy.

Prove with:
- 2x2 line graphs, with each line being a deployment strategy, and each graph representing the overall SxP combos
- Proved with a 2x2 of ROV vs each scenario line graphs; each line is a deployment type, x-axis is adoption %, y-axis being

In [ ]:
function generate_policy_deployment_graphs(csv_path::String, save_dir::String)
    𝒟 = CSV.read(csv_path, DataFrame)
    
    isdir(save_dir) || mkdir(save_dir)

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]

    policies = unique(𝒟.AdoptingPolicyCls)
    deployment_types = unique(𝒟.deployment_type)
    adoption_percentages = unique(𝒟.percent_adopt)

    fig = Figure(size = (1200, 1200))

    for (idx, policy) in enumerate(policies)
        agg_𝒟 = combine(groupby(𝒟[𝒟.AdoptingPolicyCls .== policy, :], [:percent_adopt, :deployment_type]), :value => mean)

        row = (idx - 1) ÷ 2 + 1
        col = (idx - 1) % 2 + 1
        ax = Axis(fig[row, col], title = "Victim Success Rate for $policy", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

        for deployment_type in deployment_types
            dt_data = filter(row -> row.deployment_type == deployment_type, agg_𝒟)
            x = dt_data.percent_adopt
            y = dt_data.value_mean
            lines!(ax, x, y, label = string(deployment_type))
        end

        ax.xticks = ([1e-6, 0.1, 0.2, 0.4, 0.8, 0.99], ["1e-6", "0.1", "0.2", "0.4", "0.8", "0.99"])
        if policy == "ASPA" || policy == "PeerROV"
            axislegend(ax, position = :lb, title = "Deployment Types")
        else
            axislegend(ax, position = :lt, title = "Deployment Types")
        end
    end

    save(joinpath(save_dir, "hypo8_1.png"), fig)
end

generate_policy_deployment_graphs(csv_path, "./hypo")

In [ ]:
function generate_rov_policy_graph(csv_path::String)
    𝒟 = CSV.read(csv_path, DataFrame)
    
    isdir("./hypo") || mkdir("./hypo")

    𝒟.percent_adopt .= replace(𝒟.percent_adopt, "SpecialPercentAdoptions.ONLY_ONE" => "1e-6")
    𝒟.percent_adopt = map(x -> x isa String ? parse(Float64, x) : x, 𝒟.percent_adopt)
    𝒟 = 𝒟[𝒟.outcome .== "VICTIM_SUCCESS", :]

    𝒟 = 𝒟[𝒟.AdoptingPolicyCls .== "ROV", :]

    scenarios = unique(𝒟.scenario_cls)
    deployment_types = unique(𝒟.deployment_type)
    adoption_percentages = unique(𝒟.percent_adopt)

    fig = Figure(size = (1200, 800))

    for (idx, scenario) in enumerate(scenarios)
        row = (idx - 1) ÷ 2 + 1  # Calculate the row index for subplot placement
        col = (idx - 1) % 2 + 1  # Calculate the column index for subplot placement

        agg_𝒟 = combine(groupby(𝒟[𝒟.scenario_cls .== scenario, :], [:percent_adopt, :deployment_type]), :value => mean)

        ax = Axis(fig[row, col], title = "Victim Success Rate for ROV: $scenario", xlabel = "Adoption Percentage", ylabel = "Victim Success Rate (%)")

        for deployment_type in deployment_types
            dt_data = filter(row -> row.deployment_type == deployment_type, agg_𝒟)
            x = dt_data.percent_adopt
            y = dt_data.value_mean
            lines!(ax, x, y, label = string(deployment_type))
        end

        ax.xticks = ([1e-6, 0.1, 0.2, 0.4, 0.8, 0.99], ["1e-6", "0.1", "0.2", "0.4", "0.8", "0.99"])
        if scenario == "PrefixHijack" || scenario == "SubprefixHijack"
            axislegend(ax, position = :lt, title = "Deployment Types")
        else
            axislegend(ax, position = :lb, title = "Deployment Types")
        end
    end

    save(joinpath("./hypo", "hypo8_2.png"), fig)
end

generate_rov_policy_graph(csv_path)